In [75]:
import pandas as pd 
import numpy as np
from utils import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
!ls

__pycache__       explore1.ipynb    exploring_2.ipynb utils.py


In [20]:
data = pd.read_csv('../data/kidney_disease/kidney_disease.csv')

In [21]:
data.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [22]:
data.columns

Index(['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [7]:
data['pc'].dtype

dtype('O')

In [8]:
type(data['pc'][0])

str

In [9]:
num_variables = []
categ_variables = []

In [10]:
columns = list(data.columns)
n = len(columns)
for i in range(n):
    if data[columns[i]].dtype == 'int' or data[columns[i]].dtype == 'float':
        num_variables.append(columns[i])
    else : 
        categ_variables.append(columns[i])

## Nettoyage données banknote

In [76]:
data2 = pd.read_csv('../data/banknote/data_banknote_authentication.txt')

In [77]:
data2.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [78]:
data2.columns

Index(['variance', 'skewness', 'curtosis', 'entropy', 'class'], dtype='object')

## Pre-processing

In [108]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder

Replacing missing values

In [109]:
data['wc'].dtype

dtype('float64')

In [79]:
data = clean_noisy_data(data2,classes = 2)

In [80]:
num_variables, categ_variables = detect_type(data.iloc[:,:-1])

In [81]:
num_variables

['variance', 'skewness', 'curtosis', 'entropy']

In [82]:
data_tr = replace_missing(data.iloc[:,:-1], num_variables, categ_variables)

In [83]:
data_tr.shape

(1372, 4)

In [84]:
np.array(data_tr)

array([[  3.6216 ,   8.6661 ,  -2.8073 ,  -0.44699],
       [  4.5459 ,   8.1674 ,  -2.4586 ,  -1.4621 ],
       [  3.866  ,  -2.6383 ,   1.9242 ,   0.10645],
       ...,
       [ -3.7503 , -13.4586 ,  17.5932 ,  -2.7771 ],
       [ -3.5637 ,  -8.3827 ,  12.393  ,  -1.2823 ],
       [ -2.5419 ,  -0.65804,   2.6842 ,   1.1952 ]])

In [73]:
np.concatenate(np.array(data_tr))

array([ 3.6216 ,  8.6661 , -2.8073 , ..., -0.65804,  2.6842 ,  1.1952 ])

In [85]:
data_tr = center_encode(data_tr, num_variables, categ_variables)

In [86]:
data_tr.shape

(1372, 4)

## Feature Selection

In [132]:
from sklearn.decomposition import PCA

In [160]:
pca = PCA(n_components=2)
pca.fit(data_tr)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [163]:
np.argsort(pca.components_[0])

array([21, 20, 23, 12, 18,  1,  0,  3,  6,  2,  8,  9,  7, 19, 22,  5,  4,
       10, 11, 14, 15, 17, 16, 13])

## Training

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [56]:
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
    return X_train, X_test, y_train, y_test

In [44]:
parameters = {'kernel':['linear', 'rbf', 'sigmoid', 'poly'], 'C':[1, 10], 'degree' : [2, 3],
             'gamma' : ['scale', 'auto']}
import itertools
parameters_values = list(parameters.values())
combinations = list(itertools.product(*parameters_values))
comb_parameters = []
for c in combinations : 
    d = {}
    keys = list(parameters.keys())
    for k in range(len(keys)):
        d[keys[k]] = c[k]
    comb_parameters.append(d)

In [45]:
comb_parameters

[{'kernel': 'linear', 'C': 1, 'degree': 2, 'gamma': 'scale'},
 {'kernel': 'linear', 'C': 1, 'degree': 2, 'gamma': 'auto'},
 {'kernel': 'linear', 'C': 1, 'degree': 3, 'gamma': 'scale'},
 {'kernel': 'linear', 'C': 1, 'degree': 3, 'gamma': 'auto'},
 {'kernel': 'linear', 'C': 10, 'degree': 2, 'gamma': 'scale'},
 {'kernel': 'linear', 'C': 10, 'degree': 2, 'gamma': 'auto'},
 {'kernel': 'linear', 'C': 10, 'degree': 3, 'gamma': 'scale'},
 {'kernel': 'linear', 'C': 10, 'degree': 3, 'gamma': 'auto'},
 {'kernel': 'rbf', 'C': 1, 'degree': 2, 'gamma': 'scale'},
 {'kernel': 'rbf', 'C': 1, 'degree': 2, 'gamma': 'auto'},
 {'kernel': 'rbf', 'C': 1, 'degree': 3, 'gamma': 'scale'},
 {'kernel': 'rbf', 'C': 1, 'degree': 3, 'gamma': 'auto'},
 {'kernel': 'rbf', 'C': 10, 'degree': 2, 'gamma': 'scale'},
 {'kernel': 'rbf', 'C': 10, 'degree': 2, 'gamma': 'auto'},
 {'kernel': 'rbf', 'C': 10, 'degree': 3, 'gamma': 'scale'},
 {'kernel': 'rbf', 'C': 10, 'degree': 3, 'gamma': 'auto'},
 {'kernel': 'sigmoid', 'C': 1, '

In [46]:
def determine_combinaisons(parameters):
    parameters_values = list(parameters.values())
    combinations = list(itertools.product(*parameters_values))
    comb_parameters = []
    for c in combinations : 
        d = {}
        keys = list(parameters.keys())
        for k in range(len(keys)):
            d[keys[k]] = c[k]
        comb_parameters.append(d)
    return comb_parameters

In [47]:
def training(model, parameters, X, y):
    clf = model(**parameters)
    scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')
    score = np.mean(scores)
    return score

In [48]:
def select_params(model, parameters, X, y):
    comb_parameters = determine_combinaisons(parameters)
    total_scores = []
    for i in range(len(comb_parameters)):
        total_scores.append(training(model, comb_parameters[i], X, y))
    score_max = np.max(total_scores)
    ind_max = np.argmax(total_scores)
    return comb_parameters[ind_max], score_max

In [50]:
model = SVC
parameters = {'kernel':['linear', 'rbf', 'sigmoid', 'poly'], 'C':[1, 10], 'degree' : [2, 3],
             'gamma' : ['scale', 'auto']}

In [87]:
X = data_tr
y = data.iloc[:,-1]

In [88]:
X_train, X_test, y_train, y_test = split_data(X,y)

In [89]:
select_params(model, parameters, X_train, y_train)

({'kernel': 'rbf', 'C': 10, 'degree': 2, 'gamma': 'scale'}, 1.0)

In [90]:
clf = SVC(**{'kernel':'sigmoid', 'C': 1, 'degree': 2, 'gamma': 'scale'})
np.mean(cross_val_score(clf, X_test, y_test, cv=5, scoring = 'f1_macro'))

0.78366540742504